See : https://www.kaggle.com/c/prudential-life-insurance-assessment/data


### Variable Description
Id                      A unique identifier associated with an application.

Product_Info_1-7        A set of normalized variables relating to the product applied for

Ins_Age                 Normalized age of applicant

Ht                      Normalized height of applicant

Wt                      Normalized weight of applicant

BMI                     Normalized BMI of applicant

Employment_Info_1-6     A set of normalized variables relating to the employment history of the applicant.

InsuredInfo_1-6         A set of normalized variables providing information about the applicant.

Insurance_History_1-9   A set of normalized variables relating to the insurance history of the applicant.

Family_Hist_1-5         A set of normalized variables relating to the family history of the applicant.

Medical_History_1-41    A set of normalized variables relating to the medical history of the applicant.

Medical_Keyword_1-48    A set of dummy variables relating to the presence of/absence of a medical keyword being associated with the application.

**Response**            This is the target variable, an ordinal variable relating to the final decision associated with an application


#### The following variables are all categorical (nominal) :
Product_Info_1, Product_Info_2, Product_Info_3, Product_Info_5, Product_Info_6, Product_Info_7,

Employment_Info_2, Employment_Info_3, Employment_Info_5, InsuredInfo_1, InsuredInfo_2, InsuredInfo_3,

InsuredInfo_4, InsuredInfo_5, InsuredInfo_6, InsuredInfo_7,

Insurance_History_1, Insurance_History_2, Insurance_History_3, Insurance_History_4, Insurance_History_7,

Insurance_History_8, Insurance_History_9,

Family_Hist_1,

Medical_History_2, Medical_History_3, Medical_History_4, Medical_History_5, Medical_History_6, Medical_History_7,

Medical_History_8, Medical_History_9, Medical_History_11, Medical_History_12, Medical_History_13,

Medical_History_14, Medical_History_16, Medical_History_17, Medical_History_18, Medical_History_19,

Medical_History_20, Medical_History_21, Medical_History_22, Medical_History_23, Medical_History_25,

Medical_History_26, Medical_History_27, Medical_History_28, Medical_History_29, Medical_History_30,

Medical_History_31, Medical_History_33, Medical_History_34, Medical_History_35, Medical_History_36,

Medical_History_37, Medical_History_38, Medical_History_39, Medical_History_40, Medical_History_41


#### The following variables are continuous :
Product_Info_4, Ins_Age, Ht, Wt, BMI,

Employment_Info_1, Employment_Info_4, Employment_Info_6,

Insurance_History_5,

Family_Hist_2, Family_Hist_3, Family_Hist_4, Family_Hist_5


#### The following variables are discrete :
Medical_History_1, Medical_History_10, Medical_History_15, Medical_History_24, Medical_History_32

#### The following variables are dummy variables :
Medical_Keyword_1-48

In [2]:
import math
import pandas
import numpy as np
import matplotlib.pyplot as plt

In [64]:
# machine learning
#from sklearn import datasets
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

In [4]:
# Keras
#from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, Embedding, Activation, LSTM, merge, Flatten, Dropout, Lambda
from keras.layers import RepeatVector, Reshape
from keras.models import Model, Sequential
#from keras.engine.topology import Merge
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
#from keras.layers.convolutional import *
#from keras.utils.data_utils import get_file
#
from keras import backend as K

In [5]:
import xgboost as xgb

In [6]:
from metrics import quadratic_weighted_kappa

In [7]:
df0 = pandas.read_csv("../data/train.csv.gz")

In [8]:
# WARNING : shuffle to better split the train/test sets
df = df0.sample(frac=1)

In [9]:
df['BMI_Age'] = df['BMI'] * df['Ins_Age']

med_keyword_columns = df.columns[df.columns.str.startswith('Medical_Keyword_')]
df['Med_Keywords_Count'] = df[med_keyword_columns].sum(axis=1)

In [10]:
#df.describe().transpose()

### Continuous variables
##### Product_Info_4, Ins_Age, Ht, Wt, BMI
##### Employment_Info_1, Employment_Info_4, Employment_Info_6
##### Insurance_History_5
##### Family_Hist_2, Family_Hist_3, Family_Hist_4, Family_Hist_5

In [11]:
L = []
L1 = ['Product_Info_4', 'Ins_Age', 'Ht', 'Wt', 'BMI', 'BMI_Age', 'Med_Keywords_Count',
      'Employment_Info_1', 'Employment_Info_4', 'Employment_Info_6']
L.extend(L1)
L2 = ['Insurance_History_5', 'Family_Hist_2', 'Family_Hist_3', 'Family_Hist_4', 'Family_Hist_5']
L.extend(L2)
df[L].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Product_Info_4,59381,0.328952,0.282562,0,0.076923,0.230769,0.487179,1.000000
Ins_Age,59381,0.405567,0.197190,0,0.238806,0.402985,0.567164,1.000000
Ht,59381,0.707283,0.074239,0,0.654545,0.709091,0.763636,1.000000
Wt,59381,0.292587,0.089037,0,0.225941,0.288703,0.345188,1.000000
BMI,59381,0.469462,0.122213,0,0.385517,0.451349,0.532858,1.000000
BMI_Age,59381,0.193702,0.111500,0,0.104285,0.183607,0.267714,0.805970
Med_Keywords_Count,59381,1.264765,1.480236,0,0.000000,1.000000,2.000000,16.000000
Employment_Info_1,59362,0.077582,0.082347,0,0.035000,0.060000,0.100000,1.000000
Employment_Info_4,52602,0.006283,0.032816,0,0.000000,0.000000,0.000000,1.000000
Employment_Info_6,48527,0.361469,0.349551,0,0.060000,0.250000,0.550000,1.000000


In [12]:
### note that some variables are not defined everywhere

In [13]:
L1 = ['Product_Info_4', 'Ins_Age', 'Ht', 'Wt', 'BMI','BMI_Age', 'Med_Keywords_Count']
df[L1].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Product_Info_4,59381,0.328952,0.282562,0,0.076923,0.230769,0.487179,1.00000
Ins_Age,59381,0.405567,0.197190,0,0.238806,0.402985,0.567164,1.00000
Ht,59381,0.707283,0.074239,0,0.654545,0.709091,0.763636,1.00000
Wt,59381,0.292587,0.089037,0,0.225941,0.288703,0.345188,1.00000
BMI,59381,0.469462,0.122213,0,0.385517,0.451349,0.532858,1.00000
BMI_Age,59381,0.193702,0.111500,0,0.104285,0.183607,0.267714,0.80597
Med_Keywords_Count,59381,1.264765,1.480236,0,0.000000,1.000000,2.000000,16.00000


In [14]:
for l in L:
    if not(l in L1):
        print(l, df[l].mean())
        df[l].fillna((df[l].mean()), inplace=True)

('Employment_Info_1', 0.07758209953084576)
('Employment_Info_4', 0.006282674324930456)
('Employment_Info_6', 0.36146880400148845)
('Insurance_History_5', 0.001733063699930494)
('Family_Hist_2', 0.474550064271798)
('Family_Hist_3', 0.49773737657983136)
('Family_Hist_4', 0.4448902535379512)
('Family_Hist_5', 0.48463492966550975)


In [15]:
df[L].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Product_Info_4,59381,0.328952,0.282562,0,0.076923,0.230769,0.487179,1.000000
Ins_Age,59381,0.405567,0.197190,0,0.238806,0.402985,0.567164,1.000000
Ht,59381,0.707283,0.074239,0,0.654545,0.709091,0.763636,1.000000
Wt,59381,0.292587,0.089037,0,0.225941,0.288703,0.345188,1.000000
BMI,59381,0.469462,0.122213,0,0.385517,0.451349,0.532858,1.000000
BMI_Age,59381,0.193702,0.111500,0,0.104285,0.183607,0.267714,0.805970
Med_Keywords_Count,59381,1.264765,1.480236,0,0.000000,1.000000,2.000000,16.000000
Employment_Info_1,59381,0.077582,0.082334,0,0.035000,0.060000,0.100000,1.000000
Employment_Info_4,59381,0.006283,0.030887,0,0.000000,0.000000,0.000000,1.000000
Employment_Info_6,59381,0.361469,0.315993,0,0.100000,0.350000,0.500000,1.000000


In [16]:
X = df[L].as_matrix()
Y = df['Response'].as_matrix()

In [17]:
logreg = LogisticRegression(C=1e5)
logreg.fit(X, Y)

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [18]:
# WARNING : check how Logistic handles more than 2 classes
len( [1 for y, ym in zip(Y, logreg.predict(X)) if y==ym] ) / float(len(Y))

0.42387295599602565

In [19]:
knn = KNeighborsClassifier()
knn.fit(X, Y) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [20]:
len( [1 for y, ym in zip(Y, knn.predict(X)) if y==ym] ) / float(len(Y))

0.5600444586652297

In [21]:
c2val, c2prob = chi2(X, Y)
c2val.sort()
c2val = np.fliplr([c2val])[0]
print c2val

[  1.18717250e+04   7.16825284e+02   6.06691775e+02   4.85966874e+02
   4.04887633e+02   3.46416302e+02   1.43154971e+02   4.10930796e+01
   3.18766690e+01   1.62815912e+01   1.39388607e+01   1.12703663e+01
   2.33509673e+00   1.00745802e+00   5.66677733e-01]


In [22]:
print X.shape
X_new = SelectKBest(chi2, k=2).fit_transform(X, Y)
print X_new.shape

(59381, 15)
(59381, 2)


### Turn categorical variables into dummies with OneHotEncoding

List of variables:

Product_Info_1, Product_Info_2, Product_Info_3, Product_Info_5, Product_Info_6, Product_Info_7,
Employment_Info_2, Employment_Info_3, Employment_Info_5, InsuredInfo_1, InsuredInfo_2, InsuredInfo_3,
InsuredInfo_4, InsuredInfo_5, InsuredInfo_6, InsuredInfo_7,
Insurance_History_1, Insurance_History_2, Insurance_History_3, Insurance_History_4, Insurance_History_7,
Insurance_History_8, Insurance_History_9,
Family_Hist_1,
Medical_History_2, Medical_History_3, Medical_History_4, Medical_History_5, Medical_History_6, Medical_History_7,
Medical_History_8, Medical_History_9, Medical_History_11, Medical_History_12, Medical_History_13,
Medical_History_14, Medical_History_16, Medical_History_17, Medical_History_18, Medical_History_19,
Medical_History_20, Medical_History_21, Medical_History_22, Medical_History_23, Medical_History_25,
Medical_History_26, Medical_History_27, Medical_History_28, Medical_History_29, Medical_History_30,
Medical_History_31, Medical_History_33, Medical_History_34, Medical_History_35, Medical_History_36,
Medical_History_37, Medical_History_38, Medical_History_39, Medical_History_40, Medical_History_41


In [23]:
catstring = 'Product_Info_1, Product_Info_2, Product_Info_3, Product_Info_5, Product_Info_6, Product_Info_7, '
catstring+= 'Employment_Info_2, Employment_Info_3, Employment_Info_5, '
catstring+= 'InsuredInfo_1, InsuredInfo_2, InsuredInfo_3, InsuredInfo_4, InsuredInfo_5, InsuredInfo_6, InsuredInfo_7, '
catstring+= 'Insurance_History_1, Insurance_History_2, Insurance_History_3, Insurance_History_4, Insurance_History_7, '
catstring+= 'Insurance_History_8, Insurance_History_9, '
catstring+= 'Family_Hist_1, '
catstring+= 'Medical_History_2, Medical_History_3, Medical_History_4, Medical_History_5, Medical_History_6, '
catstring+= 'Medical_History_7, Medical_History_8, Medical_History_9, Medical_History_11, Medical_History_12, '
catstring+= 'Medical_History_13, Medical_History_14, Medical_History_16, Medical_History_17, Medical_History_18, '
catstring+= 'Medical_History_19, Medical_History_20, Medical_History_21, Medical_History_22, Medical_History_23, '
catstring+= 'Medical_History_25, Medical_History_26, Medical_History_27, Medical_History_28, Medical_History_29, '
catstring+= 'Medical_History_30, Medical_History_31, Medical_History_33, Medical_History_34, Medical_History_35, '
catstring+= 'Medical_History_36, Medical_History_37, Medical_History_38, Medical_History_39, Medical_History_40, '
catstring+= 'Medical_History_41'
categories = catstring.replace(' ','').split(',')
print categories[0:10]

['Product_Info_1', 'Product_Info_2', 'Product_Info_3', 'Product_Info_5', 'Product_Info_6', 'Product_Info_7', 'Employment_Info_2', 'Employment_Info_3', 'Employment_Info_5', 'InsuredInfo_1']


In [24]:
df[categories].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Product_Info_1,59381,1.026355,0.160191,1,1,1,1,2
Product_Info_3,59381,24.415655,5.072885,1,26,26,26,38
Product_Info_5,59381,2.006955,0.083107,2,2,2,2,3
Product_Info_6,59381,2.673599,0.739103,1,3,3,3,3
Product_Info_7,59381,1.043583,0.291949,1,1,1,1,3
Employment_Info_2,59381,8.641821,4.227082,1,9,9,9,38
Employment_Info_3,59381,1.300904,0.715034,1,1,1,1,3
Employment_Info_5,59381,2.142958,0.350033,2,2,2,2,3
InsuredInfo_1,59381,1.209326,0.417939,1,1,1,1,3
InsuredInfo_2,59381,2.007427,0.085858,2,2,2,2,3


### WARNING : Product_Info_2 is not numeric

In [25]:
print( df[['Product_Info_2']].count() )
df[['Product_Info_2']].head(5)

Product_Info_2    59381
dtype: int64


,Product_Info_2
55857,D4
7958,B2
32651,D3
7293,D3
7342,D3


In [26]:
# Found at :
# https://www.kaggle.com/marcellonegro/prudential-life-insurance-assessment/xgb-offset0501/run/137585/code
df['Product_Info_2_char'] = df.Product_Info_2.str[0]
df['Product_Info_2_num'] = df.Product_Info_2.str[1]
# factorize categorical variables
df['Product_Info_2'] = pandas.factorize(df['Product_Info_2'])[0]
df['Product_Info_2_char'] = pandas.factorize(df['Product_Info_2_char'])[0]
df['Product_Info_2_num'] = pandas.factorize(df['Product_Info_2_num'])[0]
df[['Product_Info_2','Product_Info_2_char','Product_Info_2_num']].head(5)

categories.append('Product_Info_2_char')
categories.append('Product_Info_2_num')

In [27]:
encX = OneHotEncoder()
# remove Product_Info_2 as it is not numeric (should convert it separately)
#Xcat = df[categories].drop('Product_Info_2', 1).as_matrix()
Xcat = df[categories].as_matrix()
#print Xcat.shape
#print df[categories].head()
encX.fit(Xcat)
Xohe = encX.transform(Xcat).toarray()
print Xohe.shape

# as Y has 9 categories it can be usefull to treat them separately
encY = OneHotEncoder()
encY.fit(Y.reshape(-1, 1)) # reshape as Y is a vector and OHE requires a matrix
Yohe = encY.transform(Y.reshape(-1, 1))
print Yohe.shape

(59381, 842)
(59381, 8)


#### We can remove low occurence one-hot columns to reduce dimension

In [28]:
column_test = (np.sum(Xohe, axis=0) > 25) # tweak filter setting
print(np.sum(column_test*1))
Xohe_trim = Xohe[:,column_test]
print(Xohe_trim.shape)

308
(59381, 308)


### Discrete variables / WARNING still need to include these

In [29]:
discstring = 'Medical_History_1, Medical_History_10, Medical_History_15, Medical_History_24, Medical_History_32'
discretes = discstring.replace(' ', '').split(',')

missing_disc_indic = -1
for discrete in discretes:
    # WARNING : shall fill with most frequent modality ?
    df[discrete].fillna(missing_disc_indic, inplace=True)
    #df[discrete] = pandas.factorize(df[discrete])[0]
    df[discrete] = df[discrete] - missing_disc_indic # TO AVOID NEGATIVE VALUES

Xdisc = df[discretes].as_matrix()

if True:
    encD = OneHotEncoder()
    encD.fit(Xdisc)
    Xdisc_ohe = encD.transform(Xdisc).toarray()
    print Xdisc_ohe.shape

(59381, 842)


In [30]:
column_test = (np.sum(Xdisc_ohe, axis=0) > 10) # tweak filter setting
print(np.sum(column_test*1))
Xdisc_ohe_trim = Xdisc_ohe[:,column_test]
print(Xdisc_ohe_trim.shape)

379
(59381, 379)


In [31]:
df[discretes].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Medical_History_1,59381,7.620586,12.431334,0,2,4,9,241
Medical_History_10,59381,1.333086,17.216604,0,0,0,0,241
Medical_History_15,59381,31.063657,72.986443,0,0,0,0,241
Medical_History_24,59381,3.305215,23.464397,0,0,0,0,241
Medical_History_32,59381,0.241710,5.567613,0,0,0,0,241


In [32]:
df[discretes].head(10)

,Medical_History_1,Medical_History_10,Medical_History_15,Medical_History_24,Medical_History_32
55857,5,0,99,0,0
7958,0,0,0,0,0
32651,9,0,0,22,0
7293,4,0,0,0,0
7342,2,0,0,0,0
31674,13,0,0,0,0
31458,0,0,0,0,0
24261,4,0,52,0,0
18173,0,0,0,0,0
53252,0,0,0,0,0


### Dummy variables

In [33]:
dummies = ['Medical_Keyword_'+str(i) for i in range(1,49)]

In [34]:
df[dummies].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Medical_Keyword_1,59381,0.042000,0.200591,0,0,0,0,1
Medical_Keyword_2,59381,0.008942,0.094141,0,0,0,0,1
Medical_Keyword_3,59381,0.049275,0.216443,0,0,0,0,1
Medical_Keyword_4,59381,0.014550,0.119744,0,0,0,0,1
Medical_Keyword_5,59381,0.008622,0.092456,0,0,0,0,1
Medical_Keyword_6,59381,0.012597,0.111526,0,0,0,0,1
Medical_Keyword_7,59381,0.013910,0.117119,0,0,0,0,1
Medical_Keyword_8,59381,0.010407,0.101485,0,0,0,0,1
Medical_Keyword_9,59381,0.006652,0.081289,0,0,0,0,1
Medical_Keyword_10,59381,0.036459,0.187432,0,0,0,0,1


In [35]:
Xdummies = df[dummies].as_matrix()

### Merge

In [36]:
if False: # non trimmed one-hot
    Xmerge = np.concatenate((X, Xohe, Xdummies, Xdisc), axis=1)
else:
    Xmerge = np.concatenate((X, Xohe_trim, Xdummies), axis=1)
# Xdisc
# Xdisc_ohe
# Xdisc_ohe_trim ?

In [37]:
Xmerge.shape

(59381, 371)

### chi2 selection

In [44]:
def getbests(Xarray, Yarray, nbkeep=20):
    c2val, c2prob = chi2(Xarray, Yarray)
    print len([j for j, p in enumerate(c2prob) if p<0.01]) / float(len(c2prob))
    aux = c2val.tolist()
    aux.sort()
    aux.reverse()
    minc2val = aux[nbkeep]
    return [j for j, cv in enumerate(c2val) if cv>minc2val]

bests20 = getbests(Xmerge, Y, 20)
Xbests20 = Xmerge[:,bests20]
print Xbests20.shape

bests30 = getbests(Xmerge, Y, 30)
Xbests30 = Xmerge[:,bests30]
print Xbests30.shape

bests40 = getbests(Xmerge, Y, 40)
Xbests40 = Xmerge[:,bests40]
print Xbests40.shape

bests50 = getbests(Xmerge, Y, 50)
Xbests50 = Xmerge[:,bests50]
print Xbests50.shape

0.654986522911
(59381, 20)
0.654986522911
(59381, 30)
0.654986522911
(59381, 40)
0.654986522911
(59381, 50)


### XGBoost (installed from pip)
see this link :
https://www.kaggle.com/zeroblue/prudential-life-insurance-assessment/xgboost-with-optimized-offsets/code

In [48]:
columns_to_drop = ['Id', 'Response'] #, 'Medical_History_10','Medical_History_24']
xgb_num_rounds = 720
num_classes = 8
missing_indicator = -1000

def get_params():
    
    params = {}
    params["objective"] = "reg:linear"     
    params["eta"] = 0.05
    params["min_child_weight"] = 360
    params["subsample"] = 0.85
    params["colsample_bytree"] = 0.3
    params["silent"] = 1
    params["max_depth"] = 7
    plst = list(params.items())

    return plst


xgtrain = xgb.DMatrix(df.drop(columns_to_drop, axis=1), df['Response'].values, 
                        missing=missing_indicator)
#xgtest = xgb.DMatrix(test.drop(columns_to_drop, axis=1), label=test['Response'].values, 
#                        missing=missing_indicator)    

plst = get_params()

# train model
xgbmodel = xgb.train(plst, xgtrain, xgb_num_rounds) 

In [59]:
train_preds = xgbmodel.predict(xgtrain, ntree_limit=xgbmodel.best_iteration)

In [60]:
quadratic_weighted_kappa(train_preds, df['Response'].as_matrix()) # Kaggle metric

0.6284677739916824

### KNN

In [38]:
knn2 = KNeighborsClassifier()
Xknntrain = Xbests20[range(0,50000), :]
Yknntrain = Y[range(0,50000)]
Xknntest = Xbests20[range(50000,59000), :]
Yknntest = Y[range(50000,59000)]
knn2.fit(Xknntrain, Yknntrain) # lower the "bests" threshold to include more variables ... but KNN will slow drastically

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [40]:
#len( [1 for y, ym in zip(Y, knn2.predict(Xbests30)) if y==ym] ) / float(len(Y))
print knn2.score(Xknntrain, Yknntrain)
print knn2.score(Xknntest, Yknntest)

0.38398
0.350111111111


(50000,)

In [41]:
quadratic_weighted_kappa(knn2.predict(Xknntrain), Yknntrain) # Kaggle metric

0.37173411184282823

In [97]:
# split the set into different Y classes to measure their importance
np.mean(encY.transform(Yknntrain.reshape(-1, 1)).toarray(), axis=0)

array([ 0.10468,  0.11138,  0.01672,  0.02384,  0.09156,  0.18796,
        0.1364 ,  0.32746])

### SVC

In [42]:
classcol = 7
#model = LogisticRegression()
#model = KNeighborsClassifier()
#model = RandomForestClassifier(n_estimators=50)
#model = GaussianNB()
model = SVC()
Xrftrain = Xbests40[range(0,40000), :]
Yrftrain = Y[range(0,40000)]
Xrftest = Xbests40[range(40000,59000), :]
Yrftest = Y[range(40000,59000)]
colYrftrain = encY.transform(Yrftrain.reshape(-1, 1)).getcol(classcol).toarray().flatten()
colYrftest = encY.transform(Yrftest.reshape(-1, 1)).getcol(classcol).toarray().flatten()
model.fit(Xrftrain, colYrftrain)

0.7669
0.766842105263


In [44]:
print model.score(Xrftrain, colYrftrain)
print model.score(Xrftest, colYrftest)

0.7669
0.766842105263


In [43]:
quadratic_weighted_kappa(model.predict(Xrftrain), colYrftrain) # Kaggle metric

0.4662162472193002

### Random Forests

In [45]:
random_forest = RandomForestClassifier(n_estimators=50)
Xrftrain = Xbests20[range(0,50000), :]
Yrftrain = Y[range(0,50000)]
Xrftest = Xbests20[range(50000,59000), :]
Yrftest = Y[range(50000,59000)]
random_forest.fit(Xrftrain, Yrftrain)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [46]:
#Y_pred = random_forest.predict(X)
print random_forest.score(Xrftrain, Yrftrain)
print random_forest.score(Xrftest, Yrftest)

0.48264
0.429111111111


In [48]:
quadratic_weighted_kappa(random_forest.predict(Xrftrain), Yrftrain) # Kaggle metric

0.40940047759552234

### Neural Network (with Keras)

In [38]:
Xmerge.shape, Yohe.toarray().shape # WARNING : to_array

((59381, 371), (59381, 8))

In [39]:
nn_input_dim = Xmerge.shape[1]

In [40]:
if True:
    min_max_scaler = MinMaxScaler() # WARNING : is it correct for binary variables ?
    Xmerge_prepro = min_max_scaler.fit_transform(Xmerge)
else:
    std_scaler = StandardScaler().fit(Xmerge)
    Xmerge_prepro = std_scaler.transform(Xmerge)

In [41]:
Xnn_train = Xmerge_prepro[0:45000]
Xnn_valid = Xmerge_prepro[45000:]

Ynn_train = Yohe.toarray()[0:45000]
Ynn_valid = Yohe.toarray()[45000:]

### Stand alone Neural Network i.e. no mixture

In [89]:
model = Sequential()
model.add( Dense(500, init='glorot_uniform', activation='relu', input_dim=nn_input_dim) )
model.add( BatchNormalization() )
model.add( Dropout(0.4) )
model.add( Dense(200, activation='sigmoid') )
model.add( BatchNormalization() )
model.add( Dropout(0.4) )
model.add( Dense(100, activation='sigmoid') )
model.add( BatchNormalization() )
model.add( Dropout(0.4) )
model.add( Dense(8, activation='softmax') )
model.compile(optimizer=Adam(1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

In [92]:
model.optimizer.lr = 1e-4
# train 30 times (at least)
model.fit(Xnn_train, Ynn_train, nb_epoch=10, batch_size=64, validation_data=(Xnn_valid, Ynn_valid), verbose=1)

Train on 45000 samples, validate on 14381 samples
Epoch 1/10
45000/45000 [==============================] - 6s - loss: 1.2841 - acc: 0.5326 - val_loss: 1.3131 - val_acc: 0.5237
Epoch 2/10
45000/45000 [==============================] - 6s - loss: 1.2751 - acc: 0.5373 - val_loss: 1.3141 - val_acc: 0.5237
Epoch 3/10
45000/45000 [==============================] - 6s - loss: 1.2662 - acc: 0.5373 - val_loss: 1.3089 - val_acc: 0.5249
Epoch 4/10
45000/45000 [==============================] - 6s - loss: 1.2624 - acc: 0.5413 - val_loss: 1.3089 - val_acc: 0.5267
Epoch 5/10
45000/45000 [==============================] - 6s - loss: 1.2538 - acc: 0.5456 - val_loss: 1.3097 - val_acc: 0.5269
Epoch 6/10
45000/45000 [==============================] - 6s - loss: 1.2440 - acc: 0.5472 - val_loss: 1.3074 - val_acc: 0.5283
Epoch 7/10
45000/45000 [==============================] - 6s - loss: 1.2399 - acc: 0.5503 - val_loss: 1.3062 - val_acc: 0.5287
Epoch 8/10
45000/45000 [==============================] - 6s 

### GATED MIXTURE OF EXPERTS
#### A custom loss function seems tricky to implement in Keras
#### so we implement a NN that takes X,Y as input and returns the Errors as output.
#### The fit function will have a dummy null output target so that the fit minimizes the Error function.

In [113]:
NM = 2

inputs = Input(shape=(nn_input_dim,))

outputs = Input(shape=(8,))

predictions = []
for i in range(NM):
    if True:
        xi = Dense(500, init='glorot_uniform', activation='relu')(inputs)
        xi = BatchNormalization()(xi)
        xi = Dropout(0.40)(xi)
        xi = Dense(200, activation='relu')(xi)
        xi = BatchNormalization()(xi)
        xi = Dropout(0.40)(xi)
        xi = Dense(100, activation='relu')(xi)
        xi = BatchNormalization()(xi)
        xi = Dropout(0.40)(xi)
        predictions.append( Dense(8, activation='softmax')(xi) )


predmat = Reshape((NM,8))( merge(predictions, mode='concat', concat_axis=1) ) # .summary to check axis

deltas = merge([RepeatVector(NM)(outputs), predmat], output_shape=(NM,8), mode=lambda x: -(x[0] * K.log(x[1])))

deltasums = Lambda(lambda x: K.sum(x, axis=2), output_shape=lambda s: (s[0], s[1]))(deltas)# .summary to check axis

hinton_trick = True # see "Adaptive Mixtures of Local Experts"
if hinton_trick:
    Hinton1 = Lambda(lambda x: K.exp(-x), output_shape=lambda s: s)
    deltasums = Hinton1(deltasums)

gate = Dense(100, activation='relu')(inputs)
gate = BatchNormalization()(gate)
gate = Dropout(0.40)(gate)
gate = Dense(NM, activation='softmax')(gate)

errors = merge([gate, deltasums], mode='dot')
if hinton_trick:
    Hinton2 = Lambda(lambda x: -K.log(x), output_shape=lambda s: s)    
    errors = Hinton2(errors)

# a model for training only
modelG_train = Model(input=[inputs, outputs], output=errors)

predavg = merge([gate, Reshape((8,2))(predmat)], mode='dot') # WARNING : not sure about that one !

# a model for prediction / WARNING : share weights with "train" ???
modelG_pred = Model(input=inputs, output=[predavg, predmat, gate])

# INFO :
# CE is positive and we want to minimize it
# if dummy_target = 0 then MAE = Mean(CrossEntropy)
modelG_train.compile(optimizer=Adam(1e-3), loss='mean_absolute_error')

In [114]:
#modelG_train.summary() # useful when debugging tensor shapes

In [115]:
#modelG_pred.summary() # useful when debugging tensor shapes

In [122]:
modelG_train.optimizer.lr = 1e-4
# train 30 times (at least)
Yg_train_dummy = Ynn_train[:,0]*0
Yg_valid_dummy = Ynn_valid[:,0]*0
modelG_train.fit([Xnn_train, Ynn_train], Yg_train_dummy,
                 validation_data=([Xnn_valid, Ynn_valid], Yg_valid_dummy),
                 nb_epoch=5, batch_size=64, verbose=1)

Train on 45000 samples, validate on 14381 samples
Epoch 1/5
45000/45000 [==============================] - 21s - loss: 1.5650 - val_loss: 1.4164
Epoch 2/5
45000/45000 [==============================] - 22s - loss: 1.5232 - val_loss: 1.3998
Epoch 3/5
45000/45000 [==============================] - 22s - loss: 1.4943 - val_loss: 1.3731
Epoch 4/5
45000/45000 [==============================] - 22s - loss: 1.4710 - val_loss: 1.3631
Epoch 5/5
45000/45000 [==============================] - 22s - loss: 1.4458 - val_loss: 1.3465


In [66]:
# to check the mixing is not degenerate
print( np.min(modelG_pred.predict(Xnn_train[0:5000,:])[2], axis=0) )
print( np.max(modelG_pred.predict(Xnn_train[0:5000,:])[2], axis=0) )

[ 0.19560926  0.13277987]
[ 0.8672201   0.80439073]


In [67]:
# round up forecasted probabilities
(modelG_pred.predict(Xnn_train[5000:5010,:])[0]*100).astype(int)

array([[ 9,  0, 13, 27,  5,  0,  8, 37],
       [ 0,  0,  0, 65,  1,  0,  1, 59],
       [ 3, 13, 19, 22,  3,  9, 18, 27],
       [ 0,  0,  0, 67,  3,  0,  4, 53],
       [ 5,  1, 28,  2,  4,  1, 18, 26],
       [10,  3, 16, 18,  9,  7, 21, 11],
       [ 4,  2, 18, 14,  2,  5, 13, 15],
       [ 8,  2,  7, 27,  4,  2,  2, 35],
       [11,  1, 21, 17,  6,  0,  7, 38],
       [ 2,  2, 28, 15,  3,  2, 13, 30]])

In [120]:
# there is a bug with 0-th index output as CE is too large
log_loss(Ynn_train, modelG_pred.predict(Xnn_train)[0], eps=1e-3, normalize=True)

2.2577395473433866

In [121]:
# there is a bug with 0-th index output as CE is too large
-np.mean(np.log(np.sum(Ynn_train * modelG_pred.predict(Xnn_train)[0], axis=1))) # log out of sum is fine

2.195573056306388

In [123]:
preds = modelG_pred.predict(Xnn_train)

In [124]:
Ypred = np.sum(preds[1] * np.tile(np.expand_dims(preds[2],2), (1, 1, 8)), axis=1)

In [125]:
# this CE value is consistent so 1-th and 2-th index seem fine
-np.mean(np.log(np.sum(Ynn_train * Ypred, axis=1)))

1.317130311183603